# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

I. Momcheva  ->  I. Momcheva  |  ['I. Momcheva']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
G. Chauvin  ->  G. Chauvin  |  ['G. Chauvin']
Arxiv has 63 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2501.04788


extracting tarball to tmp_2501.04788... done.


/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2501.04788/00main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: '06appendix' from 'tmp_2501.04788/06appendix.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: '07Acknowledgements' from 'tmp_2501.04788/07Acknowledgements.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site

I. Momcheva  ->  I. Momcheva  |  ['I. Momcheva']


...

extracting tarball to tmp_2501.05023...

 done.


Found 65 bibliographic references in tmp_2501.05023/main.bbl.
Issues with the citations
syntax error in line 95: '=' expected
Retrieving document from  https://arxiv.org/e-print/2501.05114
extracting tarball to tmp_2501.05114...

 done.


Found 121 bibliographic references in tmp_2501.05114/paper.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2501.04788-b31b1b.svg)](https://arxiv.org/abs/2501.04788) | **The Evolution of Half-Mass Radii and Color Gradients for Young and Old Quiescent Galaxies at $0.5 < z < 3$ with JWST/PRIMER**  |
|| M. Clausen, et al. -- incl., <mark>I. Momcheva</mark> |
|*Appeared on*| *2025-01-10*|
|*Comments*| *15 pages, 9 Figures*|
|**Abstract**|            We present a study of the size growth of the red sequence between $0.5<z<3,$ tracing the evolution of quiescent galaxies in both effective half-light and half-mass radii using multi-wavelength JWST/NIRCam imaging provided by the PRIMER survey. Half-light radii are measured from imaging in 6 different filters for 455 quiescent galaxies with log($M_*/M_{\odot}$)$>10$, whereas half-mass radii are derived from the F444W profiles together with the F277W-F444W color-$M_*$/L relation. We investigate the dependence of the ratio $r_{e, \mathrm{mass}}/r_{e, \mathrm{light}}$ on redshift, stellar mass, and the wavelength used to measure $r_{e, \mathrm{light}}$, also separating the sample into younger and older quiescent galaxies. Our data demonstrate that rest-frame infrared sizes accurately trace mass-weighted sizes while sizes measured at rest-frame optical wavelengths (0.5-0.7$\mu$m) are 0.1-0.2 dex larger, with only minor variations in redshift. We find that the average size of young quiescent galaxies agrees with that of old quiescent galaxies at intermediate masses, $10<$log($M_*/M_{\odot}$)$<11$, within their respective uncertainties in all observed-frame half-light, rest-frame half-light and half-mass radius measurements. At face value, our results point to a combination of progenitor bias and minor mergers driving the size growth of intermediate-mass quiescent galaxies at $0.5<z<3$. Our results further indicate that the varying contributions to the general quiescent population by young and old quiescent galaxies can mimic evolution in redshift.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2501.05023-b31b1b.svg)](https://arxiv.org/abs/2501.05023) | **Euclid: Detecting Solar System objects in Euclid images and classifying them using Kohonen self-organising maps**  |
|| A. A. Nucita, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2025-01-10*|
|*Comments*| *Accepted for publication on Astronomy and Astrophysics. 15 Pages, 11 Figures*|
|**Abstract**|            The ESA Euclid mission will survey more than 14,000 deg$^2$ of the sky in visible and near-infrared wavelengths, mapping the extra-galactic sky to constrain our cosmological model of the Universe. Although the survey focusses on regions further than 15 deg from the ecliptic, it should allow for the detection of more than about $10^5$ Solar System objects (SSOs). After simulating the expected signal from SSOs in Euclid images acquired with the visible camera (VIS), we describe an automated pipeline developed to detect moving objects with an apparent velocity in the range of 0.1-10 arcsec/h, typically corresponding to sources in the outer Solar System (from Centaurs to Kuiper-belt objects). In particular, the proposed detection scheme is based on Sourcextractor software and on applying a new algorithm capable of associating moving objects amongst different catalogues. After applying a suite of filters to improve the detection quality, we study the expected purity and completeness of the SSO detections. We also show how a Kohonen self-organising neural network can be successfully trained (in an unsupervised fashion) to classify stars, galaxies, and SSOs. By implementing an early-stopping method in the training scheme, we show that the network can be used in a predictive way, allowing one to assign the probability of each detected object being a member of each considered class.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2501.05114-b31b1b.svg)](https://arxiv.org/abs/2501.05114) | **The ESO SupJup Survey V: Exploring Atmospheric Variability and Orbit of the Super-Jupiter AB Pictoris b with CRIRES+**  |
|| S. Gandhi, et al. -- incl., <mark>G. Chauvin</mark> |
|*Appeared on*| *2025-01-10*|
|*Comments*| *22 pages, 17 figures, 4 tables, accepted for publication in MNRAS*|
|**Abstract**|            A growing number of directly-imaged companions have been recently characterised, with robust constraints on carbon-to-oxygen ratios and even isotopic ratios. Many companions and isolated targets have also shown spectral variability. In this work we observed the super-Jupiter AB~Pictoris~b across four consecutive nights using VLT/CRIRES+ as part of the ESO SupJup survey, exploring how the constraints on chemical composition and temperature profile change over time using spectral line shape variations between nights. We performed atmospheric retrievals of the high-resolution observations and found broadly consistent results across all four nights, but there were differences for some parameters. We clearly detect H$_2$O, $^{12}$CO and $^{13}$CO in each night, but abundances varied by $\sim2\sigma$, which was correlated to the deep atmosphere temperature profiles. We also found differences in the $^{12}$C$/^{13}$C ratios in each night by up to $\sim3\sigma$, which seemed to be correlated with the cloud deck pressure. Our combined retrieval simultaneously analysing all nights together constrained broadly the average of each night individually, with the C/O$=0.59\pm0.01$, consistent with solar composition, and $^{12}$C$/^{13}$C~$ = 102\pm8$, slightly higher than the ISM and Solar System values. We also find a low projected rotational velocity, suggesting that AB~Pictoris~b is either intrinsically a slow rotator due to its young age or that the spin axis is observed pole-on with a $\sim90^\circ$ misalignment with its orbit inclination. Future observations will be able to further explore the variability and orbit of AB~Pictoris~b as well as for other companions.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2501.04917-b31b1b.svg)](https://arxiv.org/abs/2501.04917) | **Peculiar radio-bright behaviour of the Galactic black hole transient 4U 1543-47 in the 2021-2023 outburst**  |
|| <mark>X. Zhang</mark>, et al. |
|*Appeared on*| *2025-01-10*|
|*Comments*| *13 pages, 2 figures, accepted for publication in MNRAS Letters*|
|**Abstract**|            Correlated behaviours between the radio emission and the X-ray emission in Galactic black hole X-ray binaries (BH XRBs) in the X-ray hard state are crucial to the understanding of disc-jet coupling of accreting black holes. The BH transient 4U 1543-47 went into outburst in 2021 following ~19 years of quiescence. We followed it up with ~weekly cadence with MeerKAT for about one year and a half until it faded into quiescence. Multi-epoch quasi-simultaneous MeerKAT and X-ray observations allowed us to trace the compact jet emission and its X-ray emission. In its hard spectral state across three orders of magnitude of X-ray luminosities above ~10$^{34}$ ergs/s, we found the correlation between radio and X-ray emission had a power-law index of 0.82$\pm$0.09, steeper than the canonical value of ~0.6 for BH XRBs. In addition, the radio vs. X-ray correlation shows a large range of the power-law normalization, with the maximum significantly larger than that obtained for most BH XRBs, indicating it can be particularly radio-bright and variable in the X-ray binary sample. The radio emission is unlikely diluted by discrete jet components. The observed peculiar radio-bright and variable behaviours provide the evidence for the relativistic effects of a variable Lorentz factor in the range between 1 and ~2 of the compact jet.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2501.04788/./110_half_mass.jpeg', 'tmp_2501.04788/./Selections.jpeg', 'tmp_2501.04788/./color_ML_relations_277_444_detailed.jpeg']
copying  tmp_2501.04788/./110_half_mass.jpeg to _build/html/
copying  tmp_2501.04788/./Selections.jpeg to _build/html/
copying  tmp_2501.04788/./color_ML_relations_277_444_detailed.jpeg to _build/html/
exported in  _build/html/2501.04788.md
    + _build/html/tmp_2501.04788/./110_half_mass.jpeg
    + _build/html/tmp_2501.04788/./Selections.jpeg
    + _build/html/tmp_2501.04788/./color_ML_relations_277_444_detailed.jpeg
found figures ['tmp_2501.05023/./SSO_prob_map.png', 'tmp_2501.05023/./Figure_2_referee.png', 'tmp_2501.05023/./Figure_4_SOM_TRAINED_ondata.png']
copying  tmp_2501.05023/./SSO_prob_map.png to _build/html/
copying  tmp_2501.05023/./Figure_2_referee.png to _build/html/
copying  tmp_2501.05023/./Figure_4_SOM_TRAINED_ondata.png to _build/html/
exported in  _build/html/2501.05023.md
    + _build/html/tmp_2501.05023/./SSO_prob_map

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>



<div id="title">

# The Evolution of Half-Mass Radii and Color Gradients for Young and Old Quiescent Galaxies at $0.5 < z < 3$ with JWST/PRIMER

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2501.04788-b31b1b.svg)](https://arxiv.org/abs/2501.04788)<mark>Appeared on: 2025-01-10</mark> -  _15 pages, 9 Figures_

</div>
<div id="authors">

M. Clausen, et al. -- incl., <mark>I. Momcheva</mark>

</div>
<div id="abstract">

**Abstract:** We present a study of the size growth of the red sequence between $0.5<z<3,$ tracing the evolution of quiescent galaxies in both effective half-light and half-mass radii using multi-wavelength JWST/NIRCam imaging provided by the PRIMER survey.Half-light radii are measured from imaging in 6 different filters for 455 quiescent galaxies with log( $M_*/M_{\odot}$ ) $>10$ , whereas half-mass radii are derived from the F444W profiles together with the F277W-F444W color- $M_*$ /L relation.We investigate the dependence of the ratio $r_{e, \mathrm{mass}}/r_{e, \mathrm{light}}$ on redshift, stellar mass, and the wavelength used to measure $r_{e, \mathrm{light}}$ , also separating the sample into younger and older quiescent galaxies.Our data demonstrate that rest-frame infrared sizes accurately trace mass-weighted sizes while sizes measured at rest-frame optical wavelengths (0.5-0.7 $\mu$ m) are 0.1-0.2 dex larger, with only minor variations in redshift.We find that the average size of young quiescent galaxies agrees with that of old quiescent galaxies at intermediate masses, $10<$ log( $M_*/M_{\odot}$ ) $<11$ , within their respective uncertainties in all observed-frame half-light, rest-frame half-light and half-mass radius measurements.At face value, our results point to a combination of progenitor bias and minor mergers driving the size growth of intermediate-mass quiescent galaxies at $0.5<z<3$ .Our results further indicate that the varying contributions to the general quiescent population by young and old quiescent galaxies can mimic evolution in redshift.

</div>

<div id="div_fig1">

<img src="tmp_2501.04788/./110_half_mass.jpeg" alt="Fig4" width="100%"/>

**Figure 4. -** Methodology of deriving effective half-mass radiiMethodology of deriving half-mass radii. **Panels (A) and (B):** Measured SBP (black), convolved model from \texttt{GALFIT}(orange) and deconvolved model (dark blue) in two photometric bands (F227W and F444W, respectively). The residual between the flux from the original cutout and the convolved model is added to the deconvolved model (bright blue). This is considered the total flux. The lower panels display in red the magnitude of the residual correction to be small compared to the total flux. **Panel (C):** Radial color profile derived from the total flux in F277W (A) and F444W (B). **Panel (D):** Derived radial $M_*$ / $L_{444}$ profile using the color-$M_*$ / $L_{444}$ relation derived in Subsection \ref{subsec:derive_ML}. **Panel (E):** Radial luminosity profile derived from the total flux measured in F444W. **Panel (F):** Radial mass profile derived from multiplying the radial $M_*$ / $L_{444}$ profile (D) with the luminosity profile (E). **Panel (G):** Curve of growth for mass (bright green) and total flux in F444W  (dark green) with half-light and half-mass radii (dashed lines). The uncertainties of the half-mass radii are determined with Monte-Carlo simulations. (*fig:half_mass_method*)

</div>
<div id="div_fig2">

<img src="tmp_2501.04788/./Selections.jpeg" alt="Fig3" width="100%"/>

**Figure 3. -** Evolution of mass of quiescent galaxies in the PRIMER survey**Left panel :** Star-forming (gray), old quiescent (red), and young quiescent (blue) galaxies in PRIMER. Since the most massive galaxies drive the rest-frame optical size difference, we select only galaxies that have masses higher than log$M_* > 10M_{\odot}$(black dashed line). **Right panel:** Selection of quiescent galaxies (only massive log$M_* > 10M_{\odot}$ sample depicted) within (V-J)-(U-V) space (black line). The dotted line divides the sample into recently quenched (blue) and older (red) quiescent galaxies. (*fig:sample_half_mass*)

</div>
<div id="div_fig3">

<img src="tmp_2501.04788/./color_ML_relations_277_444_detailed.jpeg" alt="Fig5" width="100%"/>

**Figure 5. -** $M_*$ / $L_{444}$ dependence on redshift. Data points are color-coded by the classification of a galaxy as young quiescent (blue) or old quiescent (red). The fit parameters for each redshift bin are listed in Table \ref{tab:color_ML_fit}. (*fig:color_ML_277_444*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2501.04788"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\new}[1]{\textcolor{red}{#1}}$
$\newcommand{\newb}[1]{\textcolor{Cerulean}{#1}}$
$\newcommand{\orcid}[1]$
$\newcommand{\euc}{\textit{Euclid}\xspace}$
$\newcommand{\elvis}{\texttt{ELViS}\xspace}$
$\newcommand{\sext}{\texttt{SExtractor}\xspace}$
$\newcommand{\casu}{\texttt{CASUTools}\xspace}$
$\newcommand{\wcsfit}{\texttt{WCSFit}\xspace}$
$\newcommand{\imcore}{\texttt{Imcore}\xspace}$
$\newcommand{\ssofinder}{\texttt{SSOFinder}\xspace}$
$\newcommand{\streakdet}{\texttt{StreakDet}\xspace}$
$\newcommand{\pivot}{\textit{PIVOT}\xspace}$
$\newcommand{\tbd}[1]{\textcolor{red}{[#1]}}$
$\newcommand{\combc}[1]{\textcolor{orange}{[#1]}}$
$\newcommand{◦}{\ensuremath{^\circ}\xspace}$
$\newcommand{\arcsec}{\ensuremath{^{\prime\prime}}\xspace}$</div>



<div id="title">

# $\Euclid$: Detecting Solar System objects in $\Euclid$ images and classifying them  using Kohonen self-organising maps$\thanks{This paper is published on behalf of the Euclid Consortium.}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2501.05023-b31b1b.svg)](https://arxiv.org/abs/2501.05023)<mark>Appeared on: 2025-01-10</mark> -  _Accepted for publication on Astronomy and Astrophysics. 15 Pages, 11 Figures_

</div>
<div id="authors">

A. A. Nucita, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** The ESA $\Euclid$ mission will survey more than 14 000 deg $^2$ of the sky in visible and near-infrared wavelengths, mapping the extra-galactic sky to constrain our cosmological model of the Universe. Although the survey focusses on regions further than 15◦ from the ecliptic, it should allow for the detection of more than about $10^5$ Solar System objects (SSOs). After simulating the expected signal from SSOs in $\Euclid$ images acquired with the visible camera (VIS), we describe an automated pipeline developed to detect moving objects with an apparent velocity in the range of $0.1$ -- $10\arcsec{\rm h}^{-1}$ , typically corresponding to sources in the outer Solar System (from Centaurs to Kuiper-belt objects). In particular, the proposed detection scheme is based on $\sext$ software and on applying a new algorithm capable of associating moving objects amongst different catalogues. After applying a suite of filters to improve the detection quality, we study the expected purity and completeness of the SSO detections. We also show how a Kohonen self-organising neural network can be successfully trained (in an unsupervised fashion) to classify stars, galaxies, and SSOs. By implementing an early-stopping method in the training scheme, we show that the network can be used in a predictive way, allowing one to assign the probability of each detected object being a member of each considered class.

</div>

<div id="div_fig1">

<img src="tmp_2501.05023/./SSO_prob_map.png" alt="Fig1" width="100%"/>

**Figure 1. -** SSO probability map (for a $20\times 20$ SOM) associated with the trained neural network (see text for details). The value of each pixel (according to the associated colour bar) gives the normalised probability that the data falling in that particular neuron belongs to the SSO class. The probability is normalised so that by summing up the probability per class, one gets exactly 100\%. (*result_4-a*)

</div>
<div id="div_fig2">

<img src="tmp_2501.05023/./Figure_2_referee.png" alt="Fig9" width="100%"/>

**Figure 9. -** Performance of the SOM evaluated by calculating the average map error, $S(t)$(solid black line, _left_ panels), and number of movements, $N_{\rm move}(t)$, in a given epoch, $t$(_right_ panels). In the _left_ panels, we also give the average error of the SOM when applied to the validation input data (yellow line). The dashed vertical lines represent, for each panel, the epoch associated with the early-stopping method (see text for details).
     (*result_1*)

</div>
<div id="div_fig3">

<img src="tmp_2501.05023/./Figure_4_SOM_TRAINED_ondata.png" alt="Fig10" width="100%"/>

**Figure 10. -** 20 $\times$ 20 U-matrix of the SOM trained to classify stars, galaxies, and SSOs. Using the training data, we superimposed a different symbol and colour in the middle of each node for each type of input vector (blue circles for stars, green squares for galaxies, and red diamonds for SSOs), confirming that the SOM correctly classified the data. The dots appearing in each pixel (with a different colour for each class) indicate the number of objects of the particular class classified by the SOM. The underlying grey-scale image represents the U-matrix associated with the trained SOM, {while numbers along the axes represent the neuron position in the map.} (*result_2*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2501.05023"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$</div>



<div id="title">

# The ESO SupJup Survey V: Exploring Atmospheric Variability and Orbit of the Super-Jupiter AB Pictoris b with CRIRES+

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2501.05114-b31b1b.svg)](https://arxiv.org/abs/2501.05114)<mark>Appeared on: 2025-01-10</mark> -  _22 pages, 17 figures, 4 tables, accepted for publication in MNRAS_

</div>
<div id="authors">

S. Gandhi, et al. -- incl., <mark>G. Chauvin</mark>

</div>
<div id="abstract">

**Abstract:** A growing number of directly-imaged companions have been recently characterised, with robust constraints on carbon-to-oxygen ratios and even isotopic ratios. Many companions and isolated targets have also shown spectral variability. In this work we observed the super-Jupiter AB Pictoris b across four consecutive nights using VLT/CRIRES+ as part of the ESO SupJup survey, exploring how the constraints on chemical composition and temperature profile change over time using spectral line shape variations between nights. We performed atmospheric retrievals of the high-resolution observations and found broadly consistent results across all four nights, but there were differences for some parameters. We clearly detect $H_2$ O, $^{12}$ CO and $^{13}$ CO in each night, but abundances varied by $\sim2\sigma$ , which was correlated to the deep atmosphere temperature profiles. We also found differences in the $^{12}$ C $/^{13}$ C ratios in each night by up to $\sim3\sigma$ , which seemed to be correlated with the cloud deck pressure. Our combined retrieval simultaneously analysing all nights together constrained broadly the average of each night individually, with the C/O $=0.59\pm0.01$ , consistent with solar composition, and $^{12}$ C $/^{13}$ C $ = 102\pm8$ , slightly higher than the ISM and Solar System values. We also find a low projected rotational velocity, suggesting that AB Pictoris b is either intrinsically a slow rotator due to its young age or that the spin axis is observed pole-on with a $\sim90^\circ$ misalignment with its orbit inclination. Future observations will be able to further explore the variability and orbit of AB Pictoris b as well as for other companions.

</div>

<div id="div_fig1">

<img src="tmp_2501.05114/./CCF_med_subtracted.png" alt="Fig2" width="100%"/>

**Figure 2. -** Cross-correlation function (CCF) of the high-pass filtered data for each night subtracted by the median over all nights against the best fit model from the combined retrieval. We divide the the values by the standard deviation away from the peak and hence they act as a proxy for the signal-to-noise. Positive correlation indicates the night had higher line depths/stronger features over the others. (*fig:ccf_med_subtract*)

</div>
<div id="div_fig2">

<img src="tmp_2501.05114/./combined_nights.png" alt="Fig15" width="100%"/>

**Figure 15. -** Constraints on the parameters for the combined retrieval of AB Pic b simultaneously analysing all four nights. For the isotopic ratio, we show the Solar System value with the blue dotted line and the ISM value and 1$\sigma$ uncertainty with the red line and shaded region  ([ and Wilson 1999](https://ui.adsabs.harvard.edu/abs/1999RPPh...62..143W), [Milam, et. al 2005](https://ui.adsabs.harvard.edu/abs/2005ApJ...634.1126M)) , and for the C/O ratio we show the the solar value with the blue dotted line  ([Asplund, Amarsi and Grevesse 2021](https://ui.adsabs.harvard.edu/abs/2021A&A...653A.141A)) . The right panel shows the P-T profile and cloud deck constraints, along with the CO-$CH_4$ equilibrium, and the condensation curves for $CaTiO_3$, Fe and $Mg_2$$SiO_4$ ([Visscher, Lodders and Fegley 2010](https://ui.adsabs.harvard.edu/abs/2010ApJ...716.1060V), [Wakeford, et. al 2017](https://ui.adsabs.harvard.edu/abs/2017MNRAS.464.4247W)) . (*fig:combined_nights*)

</div>
<div id="div_fig3">

<img src="tmp_2501.05114/./obliquity.png" alt="Fig17" width="100%"/>

**Figure 17. -** (Left panel) The orbital inclination from [ and Palma-Bifani (2023)](https://ui.adsabs.harvard.edu/abs/2023A&A...670A..90P) and the updated distribution from this work in grey and light pink respectively. (Center panel) The probability density distributions for the inclination of AB Pic b for the reported period of 2.1 hour and for a period 4 and 8 times larger. (Right panel) The revised projected obliquity calculated with a period of 2.1 and 16.8 hours. On the far right, a schematic view of the system is shown, highlighting the inclination vectors of the orbit and the companion. (*fig:obliquity*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2501.05114"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

302  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

10  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

7  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
